In [1]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
from urllib import request
from urllib.request import Request, urlopen
import pandas as pd

In [2]:
def html_loader(url):
    request_site = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    webpage = urlopen(request_site).read()
    soup = BeautifulSoup(webpage,"html.parser")
    return(soup)

In [3]:
name = []
party = []
address = []
phone = []
email = []
region = []

In [4]:
for i in range(1, 34): # Max pages is 34 to iterate through the whole dataset / Also, loop starts at 1 because page 0 and 1 are the same
    
    # Url from Parliament with a list of the MPs
    url = "https://members.parliament.uk/members/commons?SearchText=&PartyId=&Gender=Any&ForParliament=0&ShowAdvanced=False&page="+str(i)
    
    # Create the Soup object from the html
    soup = html_loader(url)
    
    # Retrieve Name, Party, and Url
    primary_soup = soup.find_all('div',{"class" : "primary-info"})
    secondary_soup = soup.find_all('div',{"class" : "secondary-info"})
    region_soup = soup.find_all('div',{"class" : "indicator indicator-label"})
    url_soup = soup.find_all('a',{"class" : "card card-member"})
    url3 = 'https://members.parliament.uk'
    

    # Append Names in name list
    for row1 in primary_soup:
        name.append(row1.text.strip())
        
    # Append Parties in party list    
    for row2 in secondary_soup:
        party.append(row2.text.strip())
        
    # Append Region in region list
    
    for row3 in region_soup:
        region.append(row3.text.strip())
    
    # Retrieve Hrefs for each MP 
    for row4 in url_soup:
        urls = row4.get('href')
        
        # For each Href retrieved create a new Url and convert it into a soup object using html_loader
        # to extract Email, Phone and Address
        sophie = (url3 + urls)
        soup = html_loader(sophie)
        
        # Retrieve Email, Phone and Address
        phone_email_soup = soup.find_all('div',attrs={"class" : "col-md-7","class" : "contact-line"},limit = 2)
        address_soup = soup.find_all('div',attrs={"class" : "col-md-5"},limit = 1)
        
        
        # If phone_email_soup contains two elements then the MP has a phone and an email
        if len(phone_email_soup)==2:
            phone.append(phone_email_soup[0].text.strip())
            email.append(phone_email_soup[1].text.strip())
            
        # If phone_email_soup contains just one element, check if its phone or email, 
        # append it to the right list and append a null value to the corresponding list    
        elif len(phone_email_soup)==1:
            string= phone_email_soup[0].text.strip()
            
            if bool(string.find('Phone'))==0:
                phone.append(string)
                email.append('Email: ')
            
                
            if bool(string.find('Email'))==0:
                email.append(string)
                phone.append('Phone: ')
                
        # If phone_email_soup is empty, append nul values to both email and phone lists 
        else:
            
            phone.append('Phone: ')
            email.append('Email: ')
        
    
                
        # Append Addresses in Address list   
        address.append(address_soup[0].text.strip())

    print('Page ' + str(i)+' completed') # Counter to track the process
        

Page 1 completed
Page 2 completed
Page 3 completed
Page 4 completed
Page 5 completed
Page 6 completed
Page 7 completed
Page 8 completed
Page 9 completed
Page 10 completed
Page 11 completed
Page 12 completed
Page 13 completed
Page 14 completed
Page 15 completed
Page 16 completed
Page 17 completed
Page 18 completed
Page 19 completed
Page 20 completed
Page 21 completed
Page 22 completed
Page 23 completed
Page 24 completed
Page 25 completed
Page 26 completed
Page 27 completed
Page 28 completed
Page 29 completed
Page 30 completed
Page 31 completed
Page 32 completed
Page 33 completed


In [5]:
# Generally when phone_email_soup contains two elements, the first one is the phone and the second
# one is the email. Nontheless, in some cases its the other way around:

# Swap Email and Phone:

for counter in range(len(phone)):
    content = phone[counter]
    content2 = email[counter]
    if bool(content.find('Email'))==0 and bool(content2.find('Phone'))==0:
        email[counter]= content
        phone[counter]= content2
        
    # In some other cases there is two phones and one stays in the email list (Always with a null email)
    # Eliminate the 2nd Phone and create null email value
    elif bool(content2.find('Phone'))==0:
        email[counter]= 'Email:'
        
        

In [10]:
#Create a DateFrame with Pandas
df1 = pd.DataFrame([name, party, region, phone, email, address])
df1 = df1.transpose()
df1.columns = ['Name', 'Party', 'Region', 'Phone', 'Email', 'Address']

#Sort by Party and export to Excel
# df1.sort_values(by='Party',ascending=True,inplace=True)
df1['Phone'] = df1['Phone'].str.replace('Phone: ','')
df1['Phone'] = df1['Phone'].str.replace('Phone:','')
df2 = df1
df2['Email'] = df2['Email'].str.replace('Email: ','')
df2['Email'] = df2['Email'].str.replace('Email:','')
df3 = df2
df3.reset_index(inplace=True)
df3 = df3.drop(['index'], axis=1)
df3['Name'] = df3['Name'] + ' MP'
df4 = df3

df4.to_excel("UKGOV.xlsx",sheet_name='UKGOV') 

df4


,Name,Party,Region,Phone,Email,Address
0,Ms Diane Abbott MP,Labour,Hackney North and Stoke Newington,020 7219 4426,diane.abbott.office@parliament.uk,House of CommonsLondonSW1A 0AA
1,Debbie Abrahams MP,Labour,Oldham East and Saddleworth,020 7219 1041 Fax: 0207 219 2405,abrahamsd@parliament.uk,House of CommonsLondonSW1A 0AA
2,Nigel Adams MP,Conservative,Selby and Ainsty,020 7219 7141,nigel.adams.mp@parliament.uk,House of CommonsLondonSW1A 0AA
3,Bim Afolami MP,Conservative,Hitchin and Harpenden,,bim.afolami.mp@parliament.uk,House of CommonsLondonSW1A 0AA
4,Adam Afriyie MP,Conservative,Windsor,020 7219 8023,adam.afriyie.mp@parliament.uk,House of CommonsLondonSW1A 0AA
...,...,...,...,...,...,...
643,Sir Jeremy Wright MP,Conservative,Kenilworth and Southam,020 7219 2008,Jeremy.wright.mp@parliament.uk,House of CommonsLondonSW1A 0AA
644,Mohammad Yasin MP,Labour,Bedford,0207 219 1513,mohammad.yasin.mp@parliament.uk,House of CommonsLondonSW1A 0AA
645,Jacob Young MP,Conservative,Redcar,020 7219 3936,jacob.young.mp@parliament.uk,House of CommonsLondonSW1A 0AA
646,Nadhim Zahawi MP,Conservative,Stratford-on-Avon,020 7219 7159,nadhim.zahawi.mp@parliament.uk,House of CommonsLondonSW1A 0AA


In [227]:
df3

,index,Name,Party,Region,Phone,Email,Address
0,443,Kenny MacAskill,Alba Party,East Lothian,020 7219 6266,alan.mak.mp@parliament.uk,House of CommonsLondonSW1A 0AA
1,327,Neale Hanvey,Alba Party,Kirkcaldy and Cowdenbeath,020 7219 3000,simon.hart.mp@parliament.uk,House of CommonsLondonSW1A 0AA
2,260,Stephen Farry,Alliance,North Down,020 7219 3662,mark.fletcher.mp@parliament.uk,House of CommonsLondonSW1A 0AA
3,285,Sir Roger Gale,Conservative,North Thanet,020 7219 1759,michelle.gildernew.mp@parliament.uk,House of CommonsLondonSW1A 0AA
4,284,Mr Marcus Fysh,Conservative,Yeovil,020 7219 8131,jo.gideon.mp@parliament.uk,House of CommonsLondonSW1A 0AA
...,...,...,...,...,...,...,...
703,495,Francie Molloy,Sinn Féin,Mid Ulster,0207 219 3906,stephen.morgan.mp@parliament.uk,House of CommonsLondonSW1A 0AA
704,96,Órfhlaith Begley,Sinn Féin,West Tyrone,0207 219 3423,saqib.bhatti@parliament.uk,House of CommonsLondonSW1A 0AA
705,239,Colum Eastwood,Social Democratic & Labour Party,Foyle,0207 219 3000,Chris.Elmore.mp@parliament.uk,House of CommonsLondonSW1A 0AA
706,326,Claire Hanna,Social Democratic & Labour Party,Belfast South,0207 219 5726,sallyann.hart.mp@parliament.uk,House of CommonsLondonSW1A 0AA
